In [1]:
import json
import os
import sys
from typing import Dict, List
import torch
import random

import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration
from trl import AutoModelForCausalLMWithValueHead, AutoModelForSeq2SeqLMWithValueHead, set_seed

In [140]:
seed_val = 0
set_seed(seed_val)

In [2]:
ds = load_dataset("json", data_files="/root/autodl-tmp/BIG-Bench-Hard/bbh/web_of_lies.json",field="examples")['train']
ds_split=ds.train_test_split(test_size=0.2)
prompt_all=ds['input']
prompt_all_new= [prompt.replace('\n', ' ') for prompt in prompt_all]
prompt_all_new=['[{}] Let’ s think step by step.'.format(prompt.replace('\n', ' ')) for prompt in prompt_all_new]
answer_all=ds['target']
prompt_train=ds_split['train']['input']
prompt_test=ds_split['test']['input']
answer_test=ds_split['test']['target']
answer_train=ds_split['train']['target']



prompt_test_new= ['[{}] Let’ s think step by step.'.format(prompt.replace('\n', ' ')) for prompt in prompt_test]
prompt_train_new= ['[{}] Let’ s think step by step.'.format(prompt.replace('\n', ' ')) for prompt in prompt_train]
    

    
    

def accuracy(prompts: List[str], outputs: List[str]) -> Dict[str, List[float]]:
    match=[]
        
    for i,prompt in enumerate(prompts):

        index = prompt_all_new.index(prompt)
        if answer_all[index].lower().strip() in outputs[i].lower().strip():
            is_correct=1.0
        else:
            is_correct=0.0
                
        match.append(is_correct)

    return sum(match)/len(match)

Found cached dataset json (/root/.cache/huggingface/datasets/json/default-417f2549269fa1f4/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
    # Load the model
model_se_0 = T5ForConditionalGeneration.from_pretrained("/root/autodl-tmp/flan-t5-large")

    # Load the tokenizer
tokenizer_se = T5Tokenizer.from_pretrained("/root/autodl-tmp/flan-t5-large")

    # Create the pipeline
se_generator= pipeline("text2text-generation", model=model_se_0, tokenizer=tokenizer_se,
                        do_sample= True,
                        top_k= 10,
                        top_p= 0.99,
                        max_length=100,
                        eos_token_id= tokenizer_se.eos_token_id,
                        temperature= 1.0,
                          device=0 if int(os.environ.get("LOCAL_RANK", 0)) == 0 else -1,)

In [30]:
    # Load the model
model_se_1 = T5ForConditionalGeneration.from_pretrained("/root/autodl-tmp/output_model/web_of_lies_01epoch_5")

    # Create the pipeline
se_generator_1 = pipeline("text2text-generation", model=model_se_1, tokenizer=tokenizer_se,
                        do_sample= True,
                        top_k= 10,
                        top_p= 0.99,
                        max_length=100,
                        eos_token_id= tokenizer_se.eos_token_id,
                        temperature= 1.0,
                        device=0 if int(os.environ.get("LOCAL_RANK", 0)) == 0 else -1,)


Some weights of the model checkpoint at /root/autodl-tmp/output_model/web_of_lies_01epoch_5 were not used when initializing T5ForConditionalGeneration: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
outputs_0=[]
for i in range(len(prompt_train)):
    
    question=prompt_train[i]
    answer=answer_train[i]
    #print('££££££££££££££££££££££££££',question)
    generation=se_generator(question)[0]['generated_text']
    outputs_0.append(generation)
    #print('$$$$$$$$$$$$$',generation)
    #print('============',answer)
    #print(reward_fn(prompts=[question], outputs=[generation]))
    

In [31]:
outputs_1=[]
for i in range(len(prompt_train)):
    
    question=prompt_train[i]
    answer=answer_train[i]
    # print('££££££££££££££££££££££££££',question)
    generation=se_generator_1(question)[0]['generated_text']
    outputs_1.append(generation)
    # print('$$$$$$$$$$$$$',generation)
    #print('============',answer)
    #print(reward_fn(prompts=[question], outputs=[generation]))
    

In [28]:
accuracy_0=accuracy(prompt_train_new,outputs_0)
print(accuracy_0)

0.495


In [32]:
accuracy_1=accuracy(prompt_train_new,outputs_1)
print(accuracy_1)

0.525
